In [30]:
import xgboost as xgb
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import roc_auc_score


In [31]:
PCA = False
if PCA:
    X = pd.read_csv("PCA_dat.csv", index_col=0) # PCA Model
else:
    X = pd.read_csv('Hotel Reservations.csv') # full model
    X.drop(['booking_status', 'Booking_ID'], axis = 1, inplace = True)

y = pd.read_csv("Hotel Reservations.csv")['booking_status']

In [32]:
X = pd.get_dummies(X)

In [33]:
le = LabelEncoder()
y = pd.DataFrame(le.fit_transform(y))
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1, test_size=None, train_size=0.3)

In [34]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline(steps=[("scaler", StandardScaler()), ("clf", xgb.XGBClassifier())])


parameters = {
    'clf__learning_rate': [0.001, 0.01, 0.05, 0.1, None],
    'clf__max_depth': [2,5,7,10,15, 30, None]
}

clf = RandomizedSearchCV(estimator=pipeline,
                         param_distributions = parameters,
                         n_iter=20,
                         verbose=1,
                         cv = 5,
                         scoring = 'roc_auc')
clf_xgb = clf.fit(X_train, y_train)
print("Best parameters:", clf.best_params_)


Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best parameters: {'clf__max_depth': 10, 'clf__learning_rate': 0.1}


In [35]:
from sklearn.metrics import roc_auc_score

y_pred = clf_xgb.predict(X_test)
classifier_score_xgb = roc_auc_score(y_test, y_pred)
classifier_score_xgb

0.8616239058937933

The AUC from XGboost model is 0.8616.